In [ ]:
from ucimlrepo import fetch_ucirepo 

iris = fetch_ucirepo(id=53)

X = iris.data.features
y = iris.data.targets

default_majority = {k: 0 for k in set(y)}


In [ ]:
import numpy as math

def computeDistance(a, b):
    return math.sqrt(math.sum([(ai-bi) ** 2 for ai, bi in zip(a, b)]))


In [ ]:
def getMajorityLabel(neighboursList):
    majority = default_majority
    furthest = neighboursList[-1][0] + 1
    for dist, label in neighboursList:
        majority[label] += furthest - dist

    return max(majority, key=majority.get)


In [ ]:
def kNNClassify(X, Y, xt, k):
    distancePairList = []
    for xi, yi in X, Y:
        di = computeDistance(xi, xt)
        distancePairList.insert((di, yi))
    distancePairList.sort()

    kNeighboursList = distancePairList[:k]
    yt = getMajorityLabel(kNeighboursList)

    return yt
